### Our goal: create a neural network from scratch

In [1]:
import pandas as pd
import numpy as np
import torch
from torchvision.datasets import MNIST
import math
from functools import partial

### Get the data

In [2]:
trainset = MNIST('../', download=True, train=True)
testset = MNIST('../', download=True, train=False)

0it [00:00, ?it/s]

9920512it [00:01, 9069812.69it/s]                            


Extracting ../MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 136002.76it/s]           
0it [00:00, ?it/s]

Extracting ../MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1055035.99it/s]                            
0it [00:00, ?it/s]

Extracting ../MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 47751.31it/s]            


Extracting ../MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [3]:
# 60 000 / 10 000: classic MNIST
trainset.data.shape, testset.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [4]:
train = trainset.data.reshape(60000, -1)
test = testset.data.reshape(10000, -1)

In [5]:
y_train = trainset.targets
y_test = testset.targets

In [6]:
# FLoating point numbers work better with pytorch...
train.dtype

torch.uint8

In [7]:
train = train.to(torch.float32)
test = test.to(torch.float32)

In [8]:
# The data isn't normalized
m, std = train.mean(), train.std()
m, std

(tensor(33.3184), tensor(78.5675))

In [9]:
train = (train - m) / std
test = (test - m) / std

### define a fully connected neural net

In [10]:
def linear(x, neurons=50):
    # x is a n by m matrix; to perform matrix operations, we want a 'm by neurons' matrix
    m = x.shape[1]
    w = torch.randn(m, neurons) * math.sqrt(2/m)
    b = torch.randn(neurons)
    return x @ w + b

In [11]:
def relu(x):
    return x.clamp_min(0.)

In [12]:
def model(x):
    x = linear(x, 50)
    x = relu(x)
    x = linear(x, 10)
    return x

In [13]:
output = model(train)

In [15]:
#ten predictions for every row, sounds good
output.shape

torch.Size([60000, 10])

### define a loss function

In [18]:
def log_softmax(output):
    return (output.exp() / output.exp().sum(-1, keepdim=True)).log()

### softmax(my_tens)

In [19]:
# log-sum-exp-trick:          https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/
def logsumexp(x):
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()

def log_softmax(x): 
    return x - x.logsumexp(-1,keepdim=True)

In [20]:
def nll(output, y_train):
    return -output[range(y_train.shape[0]), y_train].mean()

In [21]:
nll(log_softmax(output), y_train)

tensor(3.3697)

### Forward pass

In [22]:
output = model(train)
print('output shape:', output.shape)
loss = nll(log_softmax(output), y_train)
print('loss', loss)

output shape: torch.Size([60000, 10])
loss tensor(4.9101)


### Backward pass

In [23]:
# parameters we can change:
EPOCHS = 35
INPUT = train
TARGETS = y_train
N_LAYERS = 2
LR = 0.3

# Automated fully connected neural net creation based on above parameters
activations = np.linspace(50, 10, N_LAYERS).astype(int)
shapes = [INPUT.shape[1]]
shapes.extend(activations)
w = []
b = []
for layer in range(N_LAYERS):
    w.append(torch.randn(shapes[layer], activations[layer]) * math.sqrt(2/shapes[layer]))
    b.append(torch.zeros(activations[layer]))
    w[-1].requires_grad_()
    b[-1].requires_grad_()
    
# Training loop
for i in range(EPOCHS):
    output = INPUT
    for layer in range(N_LAYERS-1):
        output = output @ w[layer] + b[layer]
        output = relu(output)
    
    output = output @ w[N_LAYERS-1] + b[N_LAYERS-1]
    
    loss = nll(log_softmax(output), TARGETS)
    print(loss)
    loss.backward()
    with torch.no_grad():
        for layer in range(N_LAYERS):
            w[layer] -= w[layer].grad * LR
            b[layer] -= b[layer].grad * LR
            w[layer].grad.zero_()
            b[layer].grad.zero_()

tensor(3.0576, grad_fn=<NegBackward>)
tensor(3.2114, grad_fn=<NegBackward>)
tensor(2.6823, grad_fn=<NegBackward>)
tensor(3.2061, grad_fn=<NegBackward>)
tensor(2.1392, grad_fn=<NegBackward>)
tensor(1.7504, grad_fn=<NegBackward>)
tensor(1.4585, grad_fn=<NegBackward>)
tensor(1.1633, grad_fn=<NegBackward>)
tensor(0.9743, grad_fn=<NegBackward>)
tensor(0.8709, grad_fn=<NegBackward>)
tensor(0.9000, grad_fn=<NegBackward>)
tensor(1.1877, grad_fn=<NegBackward>)
tensor(1.3111, grad_fn=<NegBackward>)
tensor(1.8775, grad_fn=<NegBackward>)
tensor(1.3656, grad_fn=<NegBackward>)
tensor(1.2602, grad_fn=<NegBackward>)
tensor(0.9392, grad_fn=<NegBackward>)
tensor(0.6839, grad_fn=<NegBackward>)
tensor(0.5855, grad_fn=<NegBackward>)
tensor(0.5395, grad_fn=<NegBackward>)
tensor(0.5132, grad_fn=<NegBackward>)
tensor(0.4953, grad_fn=<NegBackward>)
tensor(0.4870, grad_fn=<NegBackward>)
tensor(0.4905, grad_fn=<NegBackward>)
tensor(0.5177, grad_fn=<NegBackward>)
tensor(0.5813, grad_fn=<NegBackward>)
tensor(0.596

In [24]:
# even with this simple model, we see the loss decrease

### Inference

In [25]:
def accuracy(pred, targs):
    return float((pred.argmax(dim=-1) == targs).sum()) / float(len(pred))

In [26]:
output = test
for layer in range(N_LAYERS-1):
    output = output @ w[layer] + b[layer]
    output = relu(output)
    
output = output @ w[N_LAYERS-1] + b[N_LAYERS-1]
output = log_softmax(output)

In [27]:
accuracy(output, y_test)

0.8929

89% accuracy (random would be 10%,as there are 10 classes), our model is definitely learning something.
However, this code is messy and hard to maintain. In the next notebook we'll refactor it using python classes. Moreover, fully connected is a good start, but the tool of trade for computer vision is convolutions. We'll add that as well